In [8]:
#pandas version '2.2.2'
import pandas as pd
#numpy version '1.26.4'
import numpy as np

In [2]:
try:
    #google colab only - put data in a folder "/content/drive/MyDrive/capstone_data/"
    from google.colab import drive
    colab = 1
    print("Running in Google Colab, load data to \"/content/drive/MyDrive/capstone_data/\" to import it.")
    drive.mount('/content/drive')
except:
    colab = 0
    print("Load data to \"/capstone_data/\" to import it.")

Load data to "/capstone_data/" to import it.


In [3]:
if colab == 1:
    df_sen = pd.read_csv("/content/drive/MyDrive/capstone_data/1976-2020-senate.csv")
elif colab == 0:
    df_sen = pd.read_csv("capstone_data/1976-2020-senate.csv")

In [4]:
df_sen

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,2020,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,UNDER VOTES,NaN,False,total,6401,278503,False,20210114,OTHER
3625,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,KELLY LOEFFLER,REPUBLICAN,False,total,2194848,4483294,True,20210114,REPUBLICAN
3626,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,RAPHAEL WARNOCK,DEMOCRAT,False,total,2288446,4483294,True,20210114,DEMOCRAT
3627,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,False,DAVID A. PERDUE,REPUBLICAN,False,total,2213979,4483241,True,20210114,REPUBLICAN


In [ ]:
df_sen = df_sen[['year', 'state_po', 'party_simplified', 'candidatevotes', 'totalvotes']]
df_sen

,year,state_po,party_simplified,candidatevotes,totalvotes
0,1976,AZ,REPUBLICAN,321236,741210
1,1976,AZ,OTHER,1565,741210
2,1976,AZ,DEMOCRAT,400334,741210
3,1976,AZ,LIBERTARIAN,7310,741210
4,1976,AZ,OTHER,10765,741210
...,...,...,...,...,...
3624,2020,WY,OTHER,6401,278503
3625,2021,GA,REPUBLICAN,2194848,4483294
3626,2021,GA,DEMOCRAT,2288446,4483294
3627,2021,GA,REPUBLICAN,2213979,4483241


In [5]:
df_sen = df_sen[df_sen['year'] >= 2000]
df_sen = df_sen[df_sen['year'] <= 2019]

In [6]:
df_sen = df_sen[df_sen['party_simplified'] != "OTHER"]
df_sen = df_sen[df_sen['party_simplified'] != "LIBERTARIAN"]

In [9]:
df_sen_pivoted = df_sen.pivot_table(
    index=['year', 'state_po', 'totalvotes'],
    columns='party_simplified',
    values='candidatevotes',
    aggfunc='sum'
).reset_index()

df_sen_pivoted.columns.name = None
df_sen_pivoted = df_sen_pivoted.rename(columns={'DEMOCRAT': 'DEMOCRAT_votes', 'REPUBLICAN': 'REPUBLICAN_votes'})

display(df_sen_pivoted)

,year,state_po,totalvotes,DEMOCRAT_votes,REPUBLICAN_votes
0,2000,AZ,1397076,NaN,1108196.0
1,2000,CA,10623608,5932522.0,3886853.0
2,2000,CT,1311261,828902.0,448077.0
3,2000,DE,326993,181566.0,142891.0
4,2000,FL,5856731,2989487.0,2705348.0
...,...,...,...,...,...
341,2018,VT,272624,NaN,74663.0
342,2018,WA,3086168,1803364.0,1282804.0
343,2018,WI,2657841,1472914.0,1184885.0
344,2018,WV,582911,288808.0,269872.0


In [10]:
#mb moved import to top

# Create a complete list of years from 2000 to 2019
all_years = pd.DataFrame({'year': np.arange(2000, 2020)})

# Get all unique states from the original data
all_states = pd.DataFrame({'state_po': df_sen_pivoted['state_po'].unique()})

# Create a DataFrame with all combinations of years and states
all_combinations = all_years.merge(all_states, how='cross')

# Merge the complete combinations with the pivoted data
df_sen_filled = pd.merge(all_combinations, df_sen_pivoted, on=['year', 'state_po'], how='left')

# Sort by state and year before forward filling
df_sen_filled = df_sen_filled.sort_values(by=['state_po', 'year'])

# Forward fill the missing values
df_sen_filled = df_sen_filled.ffill()

# Display the filled DataFrame
display(df_sen_filled)

,year,state_po,totalvotes,DEMOCRAT_votes,REPUBLICAN_votes
34,2000,AK,NaN,NaN,NaN
84,2001,AK,NaN,NaN,NaN
134,2002,AK,229548.0,24133.0,179438.0
184,2003,AK,229548.0,24133.0,179438.0
234,2004,AK,308315.0,140424.0,149773.0
...,...,...,...,...,...
789,2015,WY,171153.0,29377.0,121554.0
839,2016,WY,171153.0,29377.0,121554.0
889,2017,WY,171153.0,29377.0,121554.0
941,2018,WY,203420.0,61227.0,136210.0


In [11]:
df_sen = df_sen_filled

In [12]:
df_sen = df_sen.rename({'year':'Year', 'state_po':'State', 'totalvotes':'Total_Sen_Votes', 'DEMOCRAT_votes':'Dem_Sen_Votes', 'REPUBLICAN_votes':'Rep_Sen_Votes'}, axis = 'columns')

In [14]:
df_sen = df_sen.sort_values(by=['Year', 'State'])

In [15]:
df_sen = df_sen.fillna(0)
display(df_sen)

,Year,State,Total_Sen_Votes,Dem_Sen_Votes,Rep_Sen_Votes
34,2000,AK,0.0,0.0,0.0
35,2000,AL,311441.0,36200.0,138149.0
36,2000,AR,2087444.0,748709.0,1335104.0
0,2000,AZ,1397076.0,400602.0,1108196.0
1,2000,CA,10623608.0,5932522.0,3886853.0
...,...,...,...,...,...
987,2019,VT,272624.0,192243.0,74663.0
988,2019,WA,3086168.0,1803364.0,1282804.0
989,2019,WI,2657841.0,1472914.0,1184885.0
990,2019,WV,582911.0,288808.0,269872.0


In [16]:
if colab == 1:
    # Export to CSV (save to Google Drive)
    df_sen.to_csv('/content/drive/My Drive/capstone_data_cleaned/df_sen.csv', index=False)
elif colab == 0:
    df_sen.to_csv('capstone_data_cleaned/df_sen.csv', index=False)